# smp

- team name: aaaaaaaaaaa
- [public leaderboard](https://www.kaggle.com/c/ift6758-a20/leaderboard): 8 (1.70612)
- Marc-Antoine Bélanger
- Eric Brunet
- Jean-Philippe Gagnon-Fleury

# project

https://github.com/ebrunet28/smp

- 20 branches
- 119 commits
- 29 pull requests
- 75 builds
- based on [whisk](https://github.com/whisk-ml/whisk) framework
- runs handled through command-line interface (with yaml files)

# dataset

In [10]:
from smp.features.features import Loader

loader = Loader()

print(loader.train.dtypes)

User Name                                       object
Personal URL                                    object
Profile Cover Image Status                      object
Profile Verification Status                     object
Profile Text Color                              object
Profile Page Color                              object
Profile Theme Color                             object
Is Profile View Size Customized?                  bool
UTC Offset                                     float64
Location                                        object
Location Public Visibility                      object
User Language                                   object
Profile Creation Timestamp                      object
User Time Zone                                  object
Num of Followers                                 int64
Num of People Following                          int64
Num of Status Updates                            int64
Num of Direct Messages                           int64
Profile Ca

# features

TODO: describe preprocessing

- boolean
    - Personal URL
    - Is Profile View Size Customized?
    - Location
- categorical
    - Profile Cover Image Status
    - Profile Verification Status
    - Location Public Visibility
    - User Language
    - User Time Zone
    - Profile Category
- discrete
    - UTC Offset
    - Num of Followers
    - Num of People Following
    - Num of Status Updates
    - Num of Direct Messages
- elapsed time
    - Profile Creation Timestamp
- float
    - Avg Daily Profile Visit Duration in seconds
    - Avg Daily Profile Clicks
- image
    - Profile Image
- rgb
    - Profile Text Color
    - Profile Page Color
    - Profile Theme Color

## boolean

### Personal URL

In [14]:
from smp.features.boolean import PersonalURL

PersonalURL().fit_transform(X=loader.train)

[Pipeline] .......... (step 1 of 2) Processing ToVector, total=   0.0s
[Pipeline] .. (step 2 of 2) Processing MissingIndicator, total=   0.0s


array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [False]])

### Is Profile View Size Customized?

### Location

## categorical

### Profile Cover Image Status

### Profile Verification Status

### Location Public Visibility

### User Language

### User Time Zone

### Profile Category

## discrete

### UTC Offset

### Num of Followers

### Num of People Following

### Num of Status Updates

### Num of Direct Messages

## elapsed time

### Profile Creation Timestamp

## float

### Avg Daily Profile Visit Duration in seconds

### Avg Daily Profile Clicks

## image

### Profile Image

## rgb

### Profile Text Color

### Profile Page Color

### Profile Theme Color

## methodology

- grid search
- models
    - random forest
    - linear regression


## model

- selected random forest
- tried linear regression


## next steps

- inspect features codependence